In [1]:
import pandas as pd
import numpy as np
import rasterio
import glob
import shutil
from tqdm import tqdm
import time
import re
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [25]:
from chu import raster2shp1
dir2 = cdir(r'P:\16000_2')
tf = glob.glob(r'P:\tif\16000_2\*.tif') 
print(tf[0])
for savename in tqdm(tf):
    raster2shp1(savename,2,dir2)

P:\tif\16000_2\ndwi1_int8_world_16164.tif


100%|████████████████████████████████████████████████████████████████████████████████| 169/169 [04:52<00:00,  1.73s/it]


In [30]:
num = 18000
dir = fr"P:\tif\{num}"
shp_list = glob.glob(fr'P:\{num}\*.shp')
#shp_list = shp_list[979:]
#tif_list=glob.glob(fr'L:\PRE_DATA\{num}\*.tif')
tif_list=glob.glob(fr'D:\zwk_data\pre_data\{num}\*.tif')
dirin = r'H:\paper\river_data\GRWL_mask_V01.01'
txt = os.path.join(dir,'log.txt')
temptxt = os.path.join(dir,'temp.txt')
dirgrwl = r"P:\tif\0\temp"
dirmatch =  cdir(os.path.join(dir, 'match'))
result_df = pd.DataFrame()

In [27]:
time.ctime()
dir = r"P:\tif\16000_2"
shp_list = glob.glob(r'P:\16000_2\*.shp')
#shp_list = shp_list[979:]
tif_list=glob.glob(r'D:\zwk_data\pre_data\16000\*.tif')

dirin = r'H:\paper\river_data\GRWL_mask_V01.01'
txt = os.path.join(dir,'log.txt')
temptxt = os.path.join(dir,'temp.txt')
dirgrwl = r"P:\tif\0\temp"
dirmatch =  cdir(os.path.join(dir, 'match'))
result_df = pd.DataFrame()

'Wed Jan 10 10:24:22 2024'

In [42]:
time.ctime()

for inshp in tqdm(shp_list):
    #pre_shp to raster
    #inshp = shp_list[i]
    intif = path_convert(inshp,tif_list)
    outtif = os.path.join(dir,os.path.split(intif)[1])
    print(intif)
    print(inshp)
    shptoraster(intif,inshp,outtif)
    
    rasterbound = extofraster(intif)
    #grwl match and river extract then to new raster 
    shp = gpd.read_file(inshp)
    
    x = (rasterbound[0]+rasterbound[2])/2
    y = (rasterbound[1]+rasterbound[3])/2
    grwl1 = grwlext(rasterbound,dirin,txt, extt = '.tif') #grwl1 is a path_list
    if len(shp) != 0:
        #grwl1 = grwl(shp,dirin,intif,txt, mod = 2, extt = '.tif') #grwl1 is a path_list

        if len(grwl1) != 0:
        #grwl match to pre
            output_raster_path2 = grwltoraster(grwl1, intif,dirgrwl,dirmatch)

            #area calculation
            sentinel = countraster(outtif)
            grwl_river = countraster(output_raster_path2)

            intxt =  f"{inshp}\t{sentinel}\t{grwl_river}\t{x}\t{y}\n"
        else:

            sentinel = countraster(outtif)
            grwl_river = 0
            intxt =  f"{inshp}\t{sentinel}\t{grwl_river}\t{x}\t{y}\n"
    else:
        print(inshp)
        print('inshp no data')
        sentinel = -1
        grwl_river = 0
        if len(grwl1) != 0:
            output_raster_path2 = grwltoraster(grwl1, intif,dirgrwl,dirmatch)
            grwl_river = countraster(output_raster_path2)
            
        intxt =  f"{inshp}\t{sentinel}\t{grwl_river}\t{x}\t{y}\n"
    with open(temptxt, 'a') as txt_file:
        txt_file.write(intxt)

copytxt = os.path.join(dir, 'area.txt')
shutil.copy(temptxt,copytxt)
time.ctime()

'Thu Jan 11 23:26:26 2024'

  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

D:\zwk_data\pre_data\1000\ndwi1_int8_world_1653.tif
P:\add\add\1000\bands_int16_1653_r.shp
['H:\\paper\\river_data\\GRWL_mask_V01.01\\SA24.tif'
 'H:\\paper\\river_data\\GRWL_mask_V01.01\\SB24.tif']


85

  8%|██████▍                                                                            | 1/13 [00:08<01:40,  8.41s/it]

...

D:\zwk_data\pre_data\1000\ndwi1_int8_world_1939.tif
P:\add\add\1000\bands_int16_1939_r.shp
['H:\\paper\\river_data\\GRWL_mask_V01.01\\SB36.tif']


84

 54%|████████████████████████████████████████████▋                                      | 7/13 [00:46<00:40,  6.70s/it]

D:\zwk_data\pre_data\1000\ndwi1_int8_world_1289.tif
P:\add\add\1000\bands_int16_1289_r.shp


RasterioIOError: D:/zwk_data/pre_data/1000/ndwi1_int8_world_1289.tif: No such file or directory

In [49]:
time.ctime()
for i in range(17,19):

    num = i* 1000
    if os.path.exists(fr'P:\add\add\{num}'):
        dir = cdir(fr"P:\add\add\{num}_2")
        shp_list = glob.glob(fr'P:\add\add\{num}\*.shp')
        #shp_list = shp_list[979:]
        #tif_list=glob.glob(fr'L:\PRE_DATA\{num}\*.tif')
        tif_list=glob.glob(fr'D:\zwk_data\pre_data\{num}\*.tif')
        dirin = r'H:\paper\river_data\GRWL_mask_V01.01'
        txt = os.path.join(dir,'log.txt')
        temptxt = os.path.join(dir,'temp.txt')
        dirgrwl = r"P:\tif\0\temp"
        dirmatch =  cdir(os.path.join(dir, 'match'))
        result_df = pd.DataFrame()

        for inshp in tqdm(shp_list):
            #pre_shp to raster
            #inshp = shp_list[i]
            intif = path_convert(inshp,tif_list)
            outtif = os.path.join(dir,os.path.split(intif)[1])
            print(intif)
            print(inshp)
            shptoraster(intif,inshp,outtif)

            rasterbound = extofraster(intif)
            #grwl match and river extract then to new raster 
            shp = gpd.read_file(inshp)

            x = (rasterbound[0]+rasterbound[2])/2
            y = (rasterbound[1]+rasterbound[3])/2
            grwl1 = grwlext(rasterbound,dirin,txt, extt = '.tif') #grwl1 is a path_list
            if len(shp) != 0:
                #grwl1 = grwl(shp,dirin,intif,txt, mod = 2, extt = '.tif') #grwl1 is a path_list

                if len(grwl1) != 0:
                #grwl match to pre
                    output_raster_path2 = grwltoraster(grwl1, intif,dirgrwl,dirmatch)

                    #area calculation
                    sentinel = countraster(outtif)
                    grwl_river = countraster(output_raster_path2)

                    intxt =  f"{inshp}\t{sentinel}\t{grwl_river}\t{x}\t{y}\n"
                else:

                    sentinel = countraster(outtif)
                    grwl_river = 0
                    intxt =  f"{inshp}\t{sentinel}\t{grwl_river}\t{x}\t{y}\n"
            else:
                print(inshp)
                print('inshp no data')
                sentinel = -1
                grwl_river = 0
                if len(grwl1) != 0:
                    output_raster_path2 = grwltoraster(grwl1, intif,dirgrwl,dirmatch)
                    grwl_river = countraster(output_raster_path2)

                intxt =  f"{inshp}\t{sentinel}\t{grwl_river}\t{x}\t{y}\n"
            with open(temptxt, 'a') as txt_file:
                txt_file.write(intxt)

        copytxt = os.path.join(dir, 'area.txt')
        shutil.copy(temptxt,copytxt)
        time.ctime()
    else:
        print(num)

'Fri Jan 12 00:37:47 2024'

  0%|                                                                                           | 0/57 [00:00<?, ?it/s]

D:\zwk_data\pre_data\17000\ndwi1_int8_world_17082.tif
P:\add\add\17000\bands_int16_17082_r.shp
H:\paper\river_data\GRWL_mask_V01.01\SE61.tif
H:\paper\river_data\GRWL_mask_V01.01\SD60.tif
H:\paper\river_data\GRWL_mask_V01.01\SD61.tif
['H:\\paper\\river_data\\GRWL_mask_V01.01\\SE60.tif']


91

  2%|█▍                                                                                 | 1/57 [00:05<05:34,  5.97s/it]

...

D:\zwk_data\pre_data\18000\ndwi1_int8_world_18609.tif
P:\add\add\18000\bands_int16_18609_r.shp
['H:\\paper\\river_data\\GRWL_mask_V01.01\\SJ59.tif'
 'H:\\paper\\river_data\\GRWL_mask_V01.01\\SJ60.tif']


93

100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [07:22<00:00,  7.63s/it]


'P:\\add\\add\\18000_2\\area.txt'

'Fri Jan 12 00:50:58 2024'

In [11]:
shp_list[471:][0]
inshp

'P:\\6000\\ndwi1_int8_world_6214.shp'

'P:\\6000\\ndwi1_int8_world_6214.shp'

In [39]:
def resnum(filename):
    # 使用正则表达式提取数字
    match = re.search(r'\d+', filename)

    if match:
        return match.group()
        #print(result)
    else:
        print("No match found.")

import os

def path_convert(inshp,tif_list):
    #series = os.path.split(inshp)[1][17:][0:-4]  #'9000'  ndwi1_int8_world
    series = os.path.split(inshp)[1][12:][0:-6]  #bands_int16_
    d = os.path.split(tif_list[0])[0] #'J:\\pre_data\\9000'
    p = os.path.split(tif_list[0])[1][0:17] #'ndwi1_int8_world_'
    
    intif = os.path.join(d,p+series+'.tif')
    return intif
    #os.path.exists(intif)

In [48]:
path_convert(inshp,tif_list)
os.path.exists(path_convert(inshp,tif_list))
inshp
intif

'D:\\zwk_data\\pre_data\\17000\\ndwi1_int8_world_1717.tif'

False

'P:\\add\\add\\17000\\bands_int16_1717_r.shp'

'D:\\zwk_data\\pre_data\\17000\\ndwi1_int8_world_1717.tif'

In [4]:
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling

def shptoraster(existing_raster_path,shapefile_path,output_raster_path, invert = True):
    # Read the existing raster to use as a template
    with rasterio.open(existing_raster_path) as src:
        # Get raster properties
        crs = src.crs
        transform = src.transform
        width = src.width
        height = src.height
        dtype = src.dtypes[0]  # Assuming a single band raster

    # Read the shapefile
    gdf = gpd.read_file(shapefile_path)
    if len(gdf) !=0:
        gdf = gdf.to_crs(crs)
        # Create an empty raster with the same properties as the existing raster
        with rasterio.open(
            output_raster_path,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=1,  # Number of bands
            dtype=dtype,
            crs=crs,
            transform=transform,
            compress = 'LZW'
        ) as dst:
            # Rasterize the geometry onto the raster
            mask = rasterio.features.geometry_mask(
                gdf.geometry,
                transform=transform,
                invert= invert,
                #all_touched = True,
                out_shape=(height, width)
            )

            # Write the rasterized data to the output raster
            dst.write(mask.astype(dtype), 1)
    else:
        print(shapefile_path)
        print('no data')
            



In [5]:
def grwlext(ext,dirin,txt, extt = '.tif'):
    '''
    gdbin: geopandas read_file
    dirin: dir stored grwl_vector
    txt: txt file to store no exist grwl vector
    mod: control return concated gdb or not, default 1 is return, otherwise, return path 
    
    import numpy as np
    import os
    import geopandas as gpd
    '''
    #ext = gdbin.geometry.total_bounds
    zimu="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    zm1 = zimu[int(abs(ext[1])//4)]
    zm2 = zimu[int(abs(ext[3])//4)]
    num1 = int((ext[0]+180)//6)+1
    num2 = int((ext[2]+180)//6)+1
    ns = 'N' if int(ext[1]//4) >= 0 else 'S'
        
    path1 = os.path.join(dirin,ns+zm1+str(num1)+extt)
    path2 = os.path.join(dirin,ns+zm1+str(num2)+extt)
                    
    ns = 'N' if int(ext[3]//4) >= 0 else 'S'
    path3 = os.path.join(dirin,ns+zm2+str(num1)+extt)
    path4 = os.path.join(dirin,ns+zm2+str(num2)+extt)
    
    if num1//10 ==0:
        path1 = os.path.join(dirin,ns+zm1+'0'+str(num1)+extt)
        path3 = os.path.join(dirin,ns+zm2+'0'+str(num1)+extt)
    if num2//10 ==0:
        path2 = os.path.join(dirin,ns+zm1+'0'+str(num2)+extt)
        path4 = os.path.join(dirin,ns+zm2+'0'+str(num2)+extt)    
    
    biao1 = [path1,path2,path3,path4]
    biao = []
    for i in range(len(biao1)):
        if os.path.exists(biao1[i]):
            biao.append(biao1[i])
        else:
            print(biao1[i])
            with open(txt,'a') as f:
                f.write(biao1[i])
    biao = np.unique(biao)
    #print('miss')
    print(biao)
    return biao


In [7]:
import chu
from chu import raster2shp1
#dirgrwl = r"P:\tif\0\temp"

    
def grwltoraster(grwl11, intif,dirgrwl,dirmatch):
    raster_paths = []
    # grwl river shp
    for i in range(len(grwl11)):
        existing_raster_path = grwl11[i]  
        output_raster_path = os.path.join(dirgrwl,os.path.split(existing_raster_path)[1])
        if not os.path.exists(output_raster_path):
            with rasterio.open(existing_raster_path) as dst:
                a = dst.read()
                profile = dst.profile
                transform = dst.transform
                crs = dst.crs
            mask = np.where((a == 255) | (a == 86) | (a == 126), 1, 0)
            with rasterio.open(output_raster_path, 'w', **profile) as dst:
                dst.write(mask)
            raster2shp1(output_raster_path,model=1,dir2 = '')
        
        # grwl river shp match with intif
        shapefile_path = os.path.splitext(output_raster_path)[0]+'.shp'
        num = os.path.split(grwl11[i])[1][0:-4]
        output_raster_path1 = os.path.join(dirmatch, num+'_'+os.path.split(intif)[1])
        
        shptoraster(intif,shapefile_path,output_raster_path1, invert = True)
        
        raster_paths.append(output_raster_path1)
        
        #merge all matched grwl river
    output_raster_path2 =  os.path.join(dirmatch, 'grwl'+'_'+os.path.split(intif)[1])
    mergeraster(raster_paths, output_raster_path2)
        
    return output_raster_path2 

In [8]:
def countraster(rasterpath, num = 1):
    with rasterio.open(rasterpath) as src:
        # 读取栅格数据
        raster_data = src.read(1)
        cnum = np.sum(raster_data == num)
    return cnum

In [9]:
def extofraster(tifpath):
    with rasterio.open(tifpath) as src:
        bounds = src.bounds
    ext = [bounds.left,bounds.bottom,bounds.right,bounds.top]
    return ext

In [10]:
def cdir(name):
    os.makedirs(name, exist_ok=True)
    return name